<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/master/notebooks/1c_retropropagacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retropropagación

En este *notebook* programaremos con NumPy una red neuronal densa y la entrenaremos para aproximar la operación XOR usando del gradiente descedente con el algoritmo de retropropagación. Recordemos que la operación XOR ($\otimes$) está de la siguiente manera:

| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |1|
|1 |0 |1|
|1 |1 |0|


In [1]:
import numpy as np

Nuestra red neuronal densa está compuesta por una capa de 2 entradas ($x_1$ y $x_2$), una capa oculta con 10 neuronas con función de activación sigmoide y una capa de salida con una sola neurona con función de activación sigmoide. Esta función de activación se define como:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

In [2]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

La función sigmoide tiene una derivada que está expresada en términos de la misma función, esto es, 

$$
\frac{\partial \sigma (z)}{\partial z} = \sigma(z) (1 - \sigma(z))
$$

In [3]:
def derivada_sigmoide(x):
    return np.multiply(sigmoide(x), (1.0 - sigmoide(x)))

Podemos ver la operación XOR como una tarea de clasificación binaria a partir de 2 entradas. Por lo tanto, usaremos la función de pérdida de entropía cruzada binaria:

$$
ECB(\mathbf{y}, \mathbf{\hat{y}})  = -\sum_{i=1}^N \left[ y^{(i)} \log \hat{y}^{(i)} + (1 - y^{(i)}) \log (1 - \hat{y}^{(i)}) \right]
$$

In [4]:
def entropia_cruzada_binaria(y, p):
    p[p == 0] = np.nextafter(0., 1.)
    p[p == 1] = np.nextafter(1., 0.)
    return -(np.log(p[y == 1]).sum() + np.log(1 - p[y == 0]).sum())

Asimismo, calcularemos la exactitud para medir el rendimiento del modelo aprendido por la red neuronal densa:

$$
exactitud = \frac{correctos}{total}
$$

In [5]:
def exactitud(y, y_predicha):
    return (y == y_predicha).mean() * 100

Ahora, definimos la función que propaga hacia adelante una entrada $\mathbf{x}^{i}$. Como la red está compuesta de 2 capas densas (1 oculta y 1 de salida), tenemos 2 matrices de pesos con sus correspondientes vectores de sesgos $\{\mathbf{W}^{\{1\}}, \mathbf{b}^{\{1\}}\}$ y $\{\mathbf{W}^{\{2\}}, \mathbf{b}^{\{2\}}\}$ de la capa oculta y la capa de salida respectivamente. Así, podemos llevar a cabo la propagación hacia adelante en esta red de la siguiente manera:

$$
	\begin{split}
				\mathbf{a}^{\{1\}} & =  \mathbf{x}^{(i)} \\
				\mathbf{z}^{\{2\}} & =  \mathbf{W}^{\{1\}} \cdot \mathbf{a}^{\{1\}} + \mathbf{b}^{\{1\}}\\
				\mathbf{a}^{\{2\}} & =  \sigma(\mathbf{z}^{\{2\}}) \\
				\mathbf{z}^{\{3\}} & =  \mathbf{W}^{\{2\}} \cdot \mathbf{a}^{\{2\}}  + \mathbf{b}^{\{2\}}\\
				\mathbf{a}^{\{3\}} & =  \sigma(\mathbf{z}^{\{3\}})\\
				\hat{y}^{(i)} & =  \mathbf{a}^{\{3\}}
			\end{split}
      $$

In [6]:
def hacia_adelante(x, W1, b1, W2, b2):
    z2 = np.dot(W1.T, x[:, np.newaxis]) + b1
    a2 = sigmoide(z2)
    z3 = np.dot(W2.T, a2) + b2
    y_hat = sigmoide(z3)
    return z2, a2, z3, y_hat

Finalmente, definimos la función para entrenar nuestra red neuronal usando gradiente descendente. Para calcular el gradiente de la función de pérdida respecto a los pesos y sesgos en cada capa empleamos el algoritmo de retropropagación.



In [7]:
def retropropagacion(X, y, alpha = 0.01, n_epocas = 100, n_ocultas = 10):
    n_ejemplos = X.shape[0]
    n_entradas = X.shape[1]
        
    # Inicialización de las matrices de pesos W y V
    W1 = np.sqrt(1.0 / n_entradas) * np.random.randn(n_entradas, n_ocultas)
    b1 = np.zeros((n_ocultas, 1))
    
    W2 = np.sqrt(1.0 / n_ocultas) * np.random.randn(n_ocultas, 1)
    b2 = np.zeros((1, 1))
    
    perdidas = np.zeros((n_epocas))
    exactitudes = np.zeros((n_epocas))
    y_predicha = np.zeros((y.shape))
    for i in range(n_epocas):
        for j in range(n_ejemplos):
            z2, a2, z3, y_hat = hacia_adelante(X[j], W1, b1, W2, b2)

            # cálculo de gradientes para W2 y b2 por retropropagación
            dz3 = y_hat - y[j]
            dW2 = np.outer(a2, dz3)
            db2 = dz3

            # cálculo de gradientes para W1 y b1 por retropropagación
            dz2 = np.dot(W2, dz3) * derivada_sigmoide(z2)
            dW1 = np.outer(X[j], dz2)
            db1 = dz2
            
            ####################################
            # IMPORTANTE 
            # la actualización de los parámetros
            # debe hacerse de forma simultánea
            W2 = W2 - alpha * dW2
            b2 = b2 - alpha * db2
            W1 = W1 - alpha * dW1
            b1 = b1 - alpha * db1

            y_predicha[j] = y_hat
            
        # calcula la pérdida en la época
        perdidas[i] = entropia_cruzada_binaria(y, y_predicha)
        exactitudes[i] = exactitud(y, np.round(y_predicha))
        print('Epoch {0}: Pérdida = {1} Exactitud = {2}'.format(i, 
                                                              perdidas[i], 
                                                              exactitudes[i]))

    return W1, W2, perdidas, exactitudes

Para probar nuestra red, generamos los ejemplos correspondientes a la operación XOR.

In [8]:
# ejemplo (XOR)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0, 1, 1, 0]]).T

Finalmente, entrenamos nuestra red con estos ejemplos por 200 épocas usando una tasa de aprendizaje $\alpha = 1.0$.

In [9]:
np.random.seed(0)
W1, W2, perdidas, exactitudes = retropropagacion(X, 
                                                 y, 
                                                 alpha = 1.0, 
                                                 n_epocas = 200,
                                                 n_ocultas = 5)

Epoch 0: Pérdida = 4.759342701710854 Exactitud = 25.0
Epoch 1: Pérdida = 4.590954054596982 Exactitud = 50.0
Epoch 2: Pérdida = 4.4120467310968845 Exactitud = 25.0
Epoch 3: Pérdida = 4.2411690549343275 Exactitud = 25.0
Epoch 4: Pérdida = 4.101747602832166 Exactitud = 25.0
Epoch 5: Pérdida = 3.99322071947935 Exactitud = 25.0
Epoch 6: Pérdida = 3.908739595255105 Exactitud = 25.0
Epoch 7: Pérdida = 3.8410611618056363 Exactitud = 25.0
Epoch 8: Pérdida = 3.785052250686191 Exactitud = 25.0
Epoch 9: Pérdida = 3.7375572526301 Exactitud = 25.0
Epoch 10: Pérdida = 3.6966571576643146 Exactitud = 25.0
Epoch 11: Pérdida = 3.6611124774461197 Exactitud = 25.0
Epoch 12: Pérdida = 3.630050227881158 Exactitud = 25.0
Epoch 13: Pérdida = 3.6028044528771344 Exactitud = 25.0
Epoch 14: Pérdida = 3.5788368437547042 Exactitud = 25.0
Epoch 15: Pérdida = 3.5576963490499685 Exactitud = 25.0
Epoch 16: Pérdida = 3.538997365528072 Exactitud = 50.0
Epoch 17: Pérdida = 3.5224068408488183 Exactitud = 50.0
Epoch 18: Pérd

Graficamos el valor de la pérdida y la exactitud en cada época para ver el comportamiento de nuestra red durante el entrenamiento:

In [10]:
import matplotlib.pyplot as plt
plt.plot(np.arange(perdidas.size), perdidas, label='ECB')
plt.plot(np.arange(exactitudes.size), exactitudes, label='Exactitud')
plt.legend()
plt.grid(True)
plt.show()

<Figure size 640x480 with 1 Axes>

## Inicializando los pesos con zeros
Como se mencionó anteriormente, las matrices de pesos $\mathbf{W^{\{1\}}}$ y $\mathbf{W^{\{2\}}}$ se initializan con valores aleatorios pequeños mientras que los vectores de sesgo $\mathbf{b^{\{1\}}}$ y $\mathbf{b^{\{1\}}}$ con zeros. Examinemos qué pasa si inicializamos las matrices de pesos con zeros. Observa los valores de los pesos en cada época.

In [11]:
def retropropagacion_zeros(X, y, alpha = 0.1, n_epocas = 100, n_ocultas = 10):
    n_ejemplos = X.shape[0]
    n_entradas = X.shape[1]
    
    # Inicializa matrices de pesos W1 y W2 y vectores de sesgos b1 y b2
    W1 = np.zeros((n_entradas, n_ocultas))
    b1 = np.zeros((n_ocultas, 1)) 
    W2 = np.zeros((n_ocultas, 1))
    b2 = np.zeros((1, 1))
    
    perdidas = np.zeros((n_epocas))
    exactitudes = np.zeros((n_epocas))
    y_predicha = np.zeros((y.shape))
    for i in range(n_epocas):
        for j in range(n_ejemplos):
            z2, a2, z3, y_hat = hacia_adelante(X[j], W1, b1, W2, b2)

            # cálculo de gradiente para W2 por retropropagación
            delta3 = (y[j] - y_hat) * derivada_sigmoide(z3)
            W2 = W2 - alpha * np.outer(a2, delta3)
            b2 = b2 - alpha * delta3
            
            # calculo de gradiente para W1 por retropropagación
            delta2 = np.dot(W2, delta3) * derivada_sigmoide(z2)
            W1 = W1 - alpha * np.outer(X[j], delta2)
            b1 = b1 - alpha * delta2
            
            y_predicha[j] = y_hat
            
        # calcula la pérdida en época
        perdidas[i] = entropia_cruzada_binaria(y, y_predicha)
        exactitudes[i] = exactitud(y, np.round(y_predicha))
        print('Epoch {0}: Pérdida = {1} Exactitud = {2}'.format(i, 
                                                              perdidas[i], 
                                                              exactitudes[i]))
        print('W1 = {0}'.format(W1))
        print('W2 = {0}'.format(W2))
            
    return W1, W2, perdidas, exactitudes

In [12]:
W1, W2, perdidas, exactitudes = retropropagacion_zeros(X, 
                                                       y, 
                                                       alpha = 1.0,
                                                       n_epocas = 5,
                                                       n_ocultas = 10)

Epoch 0: Pérdida = 2.383938269247035 Exactitud = 100.0
W1 = [[ 0.0015006   0.0015006   0.0015006   0.0015006   0.0015006   0.0015006
   0.0015006   0.0015006   0.0015006   0.0015006 ]
 [-0.00013937 -0.00013937 -0.00013937 -0.00013937 -0.00013937 -0.00013937
  -0.00013937 -0.00013937 -0.00013937 -0.00013937]]
W2 = [[0.00877009]
 [0.00877009]
 [0.00877009]
 [0.00877009]
 [0.00877009]
 [0.00877009]
 [0.00877009]
 [0.00877009]
 [0.00877009]
 [0.00877009]]
Epoch 1: Pérdida = 2.3940150790588524 Exactitud = 75.0
W1 = [[3.07843345e-03 3.07843345e-03 3.07843345e-03 3.07843345e-03
  3.07843345e-03 3.07843345e-03 3.07843345e-03 3.07843345e-03
  3.07843345e-03 3.07843345e-03]
 [6.32326402e-05 6.32326402e-05 6.32326402e-05 6.32326402e-05
  6.32326402e-05 6.32326402e-05 6.32326402e-05 6.32326402e-05
  6.32326402e-05 6.32326402e-05]]
W2 = [[0.03001629]
 [0.03001629]
 [0.03001629]
 [0.03001629]
 [0.03001629]
 [0.03001629]
 [0.03001629]
 [0.03001629]
 [0.03001629]
 [0.03001629]]
Epoch 2: Pérdida = 2.45